# Feed Forward Neural Network 실습

## Up Down Classifier 

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import csv

In [ ]:
# sample data 생성
# feature 10개의 숫자가 오름차순[0, 1, ... , 9]이면 1로 내림차순[9, 8, ..., 0]이면 0으로 labeling
# sample의 총 갯수는 train 1000개, test 100개

samples = 1000
test_samples = 100
train_dataset = './ffnn_dataset/train_dataset.csv'
test_dataset = './ffnn_dataset/test_dataset.csv'

def write_dataset(samples, test_samples, train_dir, test_dir):
    up = [i for i in range(10)]
    down = [9-i for i in range(10)]

    data = []
    label = []
    for i in range(samples):
        data.append(up)
        data.append(down)
        label.append([1])
        label.append([0])

    with open(train_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(samples-test_samples):
            writer.writerow(label[i] + data[i])
        print('train data is written')

    with open(test_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(test_samples):
            writer.writerow(label[i] + data[i])
        print('test data is written')
        
#write_dataset(1000, 100, train_dataset, test_dataset)

In [ ]:
# 이용을 강추하는 부분
train_dataset = tf.contrib.data.TextLineDataset(train_dataset)
train_dataset = train_dataset.batch(32)
#dataset = dataset.shuffle(7777)
train_dataset = train_dataset.repeat(100)

# 데코레이터!
# dataset = tf.contrib.data.TextLineDataset(train_dataset).shuffle(7777).batch(32) 
# 혹은 method 별로 끊어서 다음 라인과 같이 사용하는 것도 가능
# dataset = tf.contrib.data.TextLineDataset(train_dataset)
# dataset = dataset.batch(32)
# dataset = dataset.shuffle(7777)

In [ ]:
# test dataset도 마찬가지 방법으로 불러오기
test_dataset = tf.contrib.data.TextLineDataset(test_dataset)
test_dataset = test_dataset.batch(32)
#dataset = dataset.shuffle(7777)
test_dataset = test_dataset.repeat(99999)

In [ ]:
train_itr = train_dataset.make_one_shot_iterator()
test_itr = test_dataset.make_one_shot_iterator()

train_batch = train_itr.get_next()
test_batch = test_itr.get_next()

# record_default는 missing data 발생 시 이를 채워주는 default value를 설정하는 것
train_decoded_batch = tf.decode_csv(train_batch, record_defaults=[[0]]*11)
test_decoded_batch = tf.decode_csv(test_batch, record_defaults=[[0]]*11)

train_label = tf.reshape(train_decoded_batch[0], [-1, 1])
#label = tf.expand_dims(decoded_batch[0], axis=-1)

train_feature =  tf.stack(train_decoded_batch[1:], axis=1)
#원래의 데이터를 row로 묶어서 하나의 sample data로 만들지 col.으로 묶어서 하나의 sample로 만들지 데이터를 보고 선택!
#row_feature = tf.stack(decoded_batch[1:], axis=0)
#col_feature = tf.stack(decoded_batch[1:], axis=1)

test_label = tf.reshape(test_decoded_batch[0], [-1, 1])
test_feature =  tf.stack(test_decoded_batch[1:], axis=1)

train_label = tf.cast(train_label, tf.float32)
train_feature =tf.cast(train_feature, tf.float32)

test_label = tf.cast(test_label, tf.float32)
test_feature =tf.cast(test_feature, tf.float32)

print(train_label)
print(train_feature)
print(test_label)
print(test_feature)

In [ ]:
with tf.Session() as sess:
    _train_label, _train_feature, _test_label, _test_feature = sess.run([train_label, train_feature, test_label, test_feature])        
    # 출력하면 아래의 각 변수들은 batch size만큼의 sample data들을 가지고 있음
    print(_train_label)
    print(_train_feature)
    print(_test_label)
    print(_test_feature)
    print(_train_label.shape, _train_feature.shape)
    print(_test_label.shape, _test_feature.shape)
    
print(_train_label.dtype)

In [ ]:
# 본격적으로 모델구성

In [ ]:
# # arguments가 많으니 하나씩, 차근차근
# layer1 = tf.layers.dense(train_feature, units=5)
# layer2 = tf.layers.dense(layer1, units=6)
# layer3 = tf.layers.dense(layer2, units=7)
# layer4 = tf.layers.dense(layer3, units=8)
# out = tf.layers.dense(layer4, units=1)

# # 디버깅을 위해 layer shape을 살펴보기 위함
# print('layer1 shape {}'.format(layer1.shape))
# print('layer2 shape {}'.format(layer2.shape))
# print('layer3 shape {}'.format(layer3.shape))
# print('layer4 shape {}'.format(layer4.shape))
# print('out shape {}'.format(out.shape))


In [ ]:
# test dataset을 이용해서 학습의 loss가 줄어들고 있는지 확인 필요

In [ ]:
def bin_model(x, activation, reuse=False):
    layer1 = tf.layers.dense(x, units=5, activation=activation, reuse=reuse, name='layer1')
    layer2 = tf.layers.dense(layer1, units=6, activation=activation, reuse=reuse, name='layer2')
    layer3 = tf.layers.dense(layer2, units=7, activation=activation, reuse=reuse, name='layer3')
    layer4 = tf.layers.dense(layer3, units=8, activation=activation, reuse=reuse, name='layer4')
    out = tf.layers.dense(layer4, units=1, activation=activation, reuse=reuse, name='out')
    
    print('layer1 shape {}'.format(layer1.shape))
    print('layer2 shape {}'.format(layer2.shape))
    print('layer3 shape {}'.format(layer3.shape))
    print('layer4 shape {}'.format(layer4.shape))
    print('out shape {}'.format(out.shape))
    
    return out

    # 디버깅을 위해 layer shape을 살펴보기 위함

    
train_out = bin_model(train_feature, activation=tf.nn.sigmoid)
test_out = bin_model(test_feature, activation=tf.nn.sigmoid, reuse=True)

for v in tf.trainable_variables():
    print(v)
    
pred = tf.nn.sigmoid(test_out)

# 검증을 위한 accuracy 도출: 참고로 검증을 위한 methods는 metrics에 있음
accuracy = tf.metrics.accuracy(test_label, tf.round(pred))

loss = tf.losses.sigmoid_cross_entropy(train_label, train_out)
train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    for i in range(1000):
        _, _loss, _acc = sess.run([train_op, loss, accuracy])
        if i%100 == 0:
            _pred = sess.run(pred)
            print('step: {}, loss: {}, acc: {}'.format(i, _loss, _acc[0]))

In [1]:
import tensorflow as tf
import numpy as np
import csv

samples = 1000
test_samples = 100
train_dataset_dir = './ffnn_dataset/train_dataset.csv'
test_dataset_dir = './ffnn_dataset/test_dataset.csv'

def write_dataset(samples, test_samples, train_dir, test_dir):
    up = [i for i in range(10)]
    down = [9-i for i in range(10)]

    data = []
    label = []
    for i in range(samples):
        data.append(up)
        data.append(down)
        label.append([1])
        label.append([0])

    with open(train_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(samples-test_samples):
            writer.writerow(label[i] + data[i])
        print('train data is written')

    with open(test_dataset, 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(test_samples):
            writer.writerow(label[i] + data[i])
        print('test data is written')
        
# write_dataset(1000, 100, train_dataset, test_dataset)

train_dataset = tf.contrib.data.TextLineDataset(train_dataset_dir)
train_dataset = train_dataset.batch(32)
train_dataset = train_dataset.repeat(100)

test_dataset = tf.contrib.data.TextLineDataset(test_dataset_dir)
test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.repeat(999999)

train_itr = train_dataset.make_one_shot_iterator()
test_itr = test_dataset.make_one_shot_iterator()

train_batch = train_itr.get_next()
test_batch = test_itr.get_next()

train_batch = tf.decode_csv(train_batch, record_defaults=[[0]]*11)
test_batch = tf.decode_csv(test_batch, record_defaults=[[0]]*11)

train_label = tf.reshape(train_batch[0], [-1, 1])
# label = tf.expand_dims(train_batch[0], axis=-1)
train_feature = tf.stack(train_batch[1:], axis=1)

test_label = tf.reshape(test_batch[0], [-1, 1])
# label = tf.expand_dims(test_batch[0], axis=-1)
test_feature = tf.stack(test_batch[1:], axis=1)

train_label = tf.cast(train_label, tf.float32)
train_feature = tf.cast(train_feature, tf.float32)
test_label = tf.cast(test_label, tf.float32)
test_feature = tf.cast(test_feature, tf.float32)

def bin_model(x, activation=None, reuse=False):
    layer1 = tf.layers.dense(x, units=5, activation=activation, reuse=reuse, name='layer1')
    layer2 = tf.layers.dense(layer1, units=6, activation=activation, reuse=reuse, name='layer2')
    layer3 = tf.layers.dense(layer2, units=7, activation=activation, reuse=reuse, name='layer3')
    layer4 = tf.layers.dense(layer3, units=8, activation=activation, reuse=reuse, name='layer4')
    out = tf.layers.dense(layer4, units=1, reuse=reuse, name='out')
    print('layer1 shape {}'.format(layer1.shape))
    print('layer2 shape {}'.format(layer2.shape))
    print('layer3 shape {}'.format(layer3.shape))
    print('layer4 shape {}'.format(layer4.shape))
    print('out shape {}'.format(out.shape))
    return out

train_out = bin_model(train_feature, activation=tf.nn.sigmoid) 
test_out = bin_model(test_feature, activation=tf.nn.sigmoid, reuse=True)

for v in tf.trainable_variables():
    print(v)
    
# variable이 모두 define되는 시점에서 saver를 만든다.

saver = tf.train.Saver()



pred = tf.nn.sigmoid(test_out)
accuracy = tf.metrics.accuracy(test_label, tf.round(pred))

loss = tf.losses.sigmoid_cross_entropy(train_label, train_out)
train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)

tf.summary.scalar('loss', loss)

merged = tf.summary.merge_all()

/Users/beomyongnho/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Instructions for updating:
Use `tf.data.TextLineDataset`.
layer1 shape (?, 5)
layer2 shape (?, 6)
layer3 shape (?, 7)
layer4 shape (?, 8)
out shape (?, 1)
layer1 shape (?, 5)
layer2 shape (?, 6)
layer3 shape (?, 7)
layer4 shape (?, 8)
out shape (?, 1)
<tf.Variable 'layer1/kernel:0' shape=(10, 5) dtype=float32_ref>
<tf.Variable 'layer1/bias:0' shape=(5,) dtype=float32_ref>
<tf.Variable 'layer2/kernel:0' shape=(5, 6) dtype=float32_ref>
<tf.Variable 'layer2/bias:0' shape=(6,) dtype=float32_ref>
<tf.Variable 'layer3/kernel:0' shape=(6, 7) dtype=float32_ref>
<tf.Variable 'layer3/bias:0' shape=(7,) dtype=float32_ref>
<tf.Variable 'layer4/kernel:0' shape=(7, 8) dtype=float32_ref>
<tf.Variable 'layer4/bias:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'out/kernel:0' shape=(8, 1) dtype=float32_ref>
<tf.Variable 'out/bias:0' shape=(1,) dtype=float32_ref>


In [2]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    writer = tf.summary.FileWriter('./ffnn_logs/', sess.graph)
    for i in range(1000):
        _, _loss, _acc, _summ = sess.run([train_op, loss, accuracy, merged])
        writer.add_summary(_summ, i)
        if i%100 == 0:
            _pred = sess.run(pred)
            print('step: {}, loss: {}, acc: {}'.format(i, _loss, _acc[0]))
    saver.save(sess, './ffnn_logs/ffnn')

step: 0, loss: 0.6946947574615479, acc: 0.0
step: 100, loss: 0.6933138370513916, acc: 0.5
step: 200, loss: 0.6931241750717163, acc: 0.5
step: 300, loss: 0.693089485168457, acc: 0.5
step: 400, loss: 0.6930748820304871, acc: 0.5
step: 500, loss: 0.6930627822875977, acc: 0.5
step: 600, loss: 0.6930509805679321, acc: 0.5
step: 700, loss: 0.693039059638977, acc: 0.5
step: 800, loss: 0.6930269598960876, acc: 0.5
step: 900, loss: 0.6930146217346191, acc: 0.5


In [3]:
with tf.Session() as sess:
    saver.restore(sess, './ffnn_logs/ffnn')
    _pred = sess.run(pred)
    print(_pred)

INFO:tensorflow:Restoring parameters from ./ffnn_logs/ffnn
[[ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]
 [ 0.49987361]
 [ 0.49972823]]


In [ ]:
# tensorboard 실행을 위한 쉘 명령어
# tensorboard -logdir=/path/of/your/logs #1.4.1버젼
# tensorboard --logdir=/path/of/your/logs #1.3.1버젼

In [4]:
# 실행할 때마다 그래프가 계속 생성되는 것을 방지하기 위해서는 다음 코드를 사용하여 리셋
tf.reset_default_graph()